**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [20]:
!pip install sk-video

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import cv2
import json
import skvideo.io
import skvideo


from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function <em>act</em> selects an action via the policy of the agent with probability 1 - $\epsilon$ and otherwise chooses a random action. The value parameter $\epsilon$ determines how <em>greedy</em> the agent behaves. It is very important to find a good <strong><em>tradeoff</strong></em> between <em>exploration</em> and <em>greedyness</em>. A too high value of $\epsilon$ means that the agent will most of the time choose the best option he has discovered so far, instead of finding unexplored options that might be better. On the other side, if one selects actions at random too often, the agent will partialy lose the ability to look for positive rewards.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time + 2, grid_size * self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array <em> self.board </em> contains the rewards of each cells in the island as well as 0 values outside the island's initial range to handle the rat visibiliy of 2 cells. At coordinate [ i , j ], the value corresponds to +0.5 if there is a cheese on cell [ i , j ] of the island, -1 if there is poison, 0 if the cell is empty and 0 as well on cells outside the island.  Once the rat has been to a position [ i , j ] the value at coordinate [ i , j ] of <em> self.board </em> is set to zero so that the rat can only enjoy the reward of each cell once.

The array <em> self.position </em> keeps track of the position of the rat. It has also been extended to handle the vision range of the rat. At coordinate [ i , j ] , the value corresponds to 1 if the rat is at position [ i , j ], -1 for coordinates outside the island's range and 0 otherwise.




## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(self.n_action)      

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Get the reaction of the environment to the action
            prev_state = state
            state, reward, game_over = env.act(action)
            
            # Update gains and losses
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({:.1f})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [60]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 0/7.0. Average score (-7.0)
Win/lose count 2.5/2.0. Average score (-3.25)
Win/lose count 3.0/1.0. Average score (-1.5)
Win/lose count 3.0/4.0. Average score (-1.375)
Win/lose count 1.5/2.0. Average score (-1.2)
Win/lose count 3.0/3.0. Average score (-1.0)
Win/lose count 0.5/3.0. Average score (-1.2142857142857142)
Win/lose count 1.0/3.0. Average score (-1.3125)
Win/lose count 2.5/1.0. Average score (-1.0)
Win/lose count 2.5/4.0. Average score (-1.05)
Final score: -1.05


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




(1) Let \(\pi\) be a policy, the state-value function satisfies the
bellman equation:
\[Q^{\pi}(s,a) = E_{p^{\pi}}[\sum_{t=0}^{\infty} \gamma^{t} r(s_{t},a_{t}) \: | \: s_{0} = s, a_{0} = a]\\
 = E_{p^{\pi}}[ r(s,a) + \sum_{t=1}^{\infty} \gamma^{t} r(s_{t}, a_{t}) \: | \: s_{0} = s, a_{0} = a]\\
  = r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty} \gamma^{t} r(s_{t+1},a_{t+1}) \: | \: s_{0} = s, a_{0} = a]\\ = r(s,a) + \gamma \sum_{s' \in S} p(s' | s,a) E_{p^{\pi}}[ \sum_{t=0}^{\infty} \gamma^{t} r(s_{t},a_{t}) | s_{0} = s', a_{0}=a']\\
= r(s,a) + \gamma \sum_{s' \in S} p(s' | s,a) Q^{\pi}(s',a') = E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]\]

(2) Now let \(\pi^{*}\) be the optimal policy, using the result above,
we obtain:
\[Q^{*}(s,a) = E_{(s',a')\sim p^{\pi^{*}}(.|s,a)}[r(s,a)+\gamma Q^{\pi^{*}}(s',a')]\\
=  E_{s'\sim \pi^{*}(.|s,a)}[r(s,a)+\gamma  Q^{\pi^{*}}(s',\pi^{*}(s'))]\\
= E_{s'\sim \pi^{*}(.|s,a)}[r(s,a)+\gamma \max_{a'}Q^{\pi^{*}}(s',a')]\\
= E_{s'\sim \pi^{*}(.|s,a)}[r(s,a)+\gamma \max_{a'}Q^{*}(s',a')]\\\]

(3) From the last part, the optimal policy \(\pi^{*}\) satisfies:
\[E_{s'\sim \pi^{*}(.|s,a)}[r(s,a)+\gamma \max_{a'}Q^{*}(s',a')-Q^{*}(s,a)] = 0\\\]
Hence, a natural idea is to take the norm of this quantitiy as the loss





***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m) # add new element
        if len(self.memory) > self.max_memory:
            del self.memory[0]  # delete oldest element in the memory if memory full
         

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]))

        
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        # Memorize the states
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        nx_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        # Build the batch used for training
        for i in range(self.batch_size):
            # We Choose randomly from the memory
            s_, n_s_, a_, r_, game_over = self.memory.random_access()

            # We fill the target_q using our predictions
            input_states[i,:,:,:] = s_
            nx_states[i,:,:,:] = n_s_
            state = s_.reshape(1, 5, 5, self.n_state)
            target_q[i,:] = self.model.predict(state)


            if not game_over_:
                nx_state = n_s_.reshape(1,5, 5, self.n_state)
                predictions = self.model.predict(nx_state)
                target_q[i, a_] = r_ + self.discount * max(predictions[0,:])
            else:
                target_q[i,a_] = r_


        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        input_shape = (5, 5, self.n_state)
        
        model = Sequential()
        model.add(Flatten(input_shape= input_shape))
        model.add((Dense(32, activation='relu')))
        model.add((Dense(16, activation='relu')))
        model.add(Dense(self.n_action, activation=None))

      
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [80]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.0002, epsilon = 0.2, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/020 | Loss 0.0481 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/020 | Loss 0.0414 | Win/lose count 7.0/9.0 (-2.0)
Epoch 002/020 | Loss 0.0323 | Win/lose count 6.0/8.0 (-2.0)
Epoch 003/020 | Loss 0.0623 | Win/lose count 5.0/12.0 (-7.0)
Epoch 004/020 | Loss 0.0250 | Win/lose count 4.0/6.0 (-2.0)
Epoch 005/020 | Loss 0.0398 | Win/lose count 6.0/9.0 (-3.0)
Epoch 006/020 | Loss 0.0502 | Win/lose count 3.0/3.0 (0.0)
Epoch 007/020 | Loss 0.0768 | Win/lose count 4.5/7.0 (-2.5)
Epoch 008/020 | Loss 0.0450 | Win/lose count 4.0/3.0 (1.0)
Epoch 009/020 | Loss 0.0557 | Win/lose count 6.5/9.0 (-2.5)
Epoch 010/020 | Loss 0.0218 | Win/lose count 2.0/1.0 (1.0)
Epoch 011/020 | Loss 0.0541 | Win/lose count 5.0/5.0 (0.0)
Epoch 012/020 | Loss 0.0463 | Win/lose count 5.5/6.0 (-0.5)
Epoch 013/020 | Loss 0.0382 | Win/lose count 7.5/9.0 (-1.5)
Epoch 014/020 | Loss 0.0489 | Win/lose count 4.5/4.0 (0.5)
Epoch 015/020 | Loss 0.0661 | Win/lose count 5.0/8.0 (-3.0)
Epoch 016/020 | Loss 0.0422 | Win/lose count

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)

        input_shape = (5, 5, self.n_state)
        
        # Model
        model = Sequential()
        model.add(Conv2D(32, 3, input_shape=input_shape, activation='relu', padding="same"))
        model.add(Conv2D(64, 3, activation='relu', padding="same"))
        model.add(Conv2D(32, 3, activation='relu', padding="same"),)
        model.add(Flatten())
        model.add(Dense(self.n_action, activation=None))
     
        
        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [83]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.0002, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/020 | Loss 0.0024 | Win/lose count 4.0/6.0 (-2.0)
Epoch 001/020 | Loss 0.0028 | Win/lose count 2.0/1.0 (1.0)
Epoch 002/020 | Loss 0.0029 | Win/lose count 1.5/3.0 (-1.5)
Epoch 003/020 | Loss 0.0125 | Win/lose count 6.5/7.0 (-0.5)
Epoch 004/020 | Loss 0.0109 | Win/lose count 3.0/0 (3.0)
Epoch 005/020 | Loss 0.0029 | Win/lose count 5.5/5.0 (0.5)
Epoch 006/020 | Loss 0.0028 | Win/lose count 1.5/8.0 (-6.5)
Epoch 007/020 | Loss 0.0206 | Win/lose count 6.0/5.0 (1.0)
Epoch 008/020 | Loss 0.0026 | Win/lose count 2.5/5.0 (-2.5)
Epoch 009/020 | Loss 0.0151 | Win/lose count 2.5/4.0 (-1.5)
Epoch 010/020 | Loss 0.0198 | Win/lose count 3.0/2.0 (1.0)
Epoch 011/020 | Loss 0.0098 | Win/lose count 5.0/5.0 (0.0)
Epoch 012/020 | Loss 0.0016 | Win/lose count 3.5/4.0 (-0.5)
Epoch 013/020 | Loss 0.0210 | Win/lose count 3.0/5.0 (-2.0)
Epoch 014/020 | Loss 0.0045 | Win/lose count 2.0/3.0 (-1.0)
Epoch 015/020 | Loss 0.0123 | Win/lose count 3.5/4.0 (-0.5)
Epoch 016/020 | Loss 0.0112 | Win/lose count 4.5

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [87]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.0/2.0. Average score (-1.0)
Win/lose count 1.5/4.0. Average score (-1.75)
Win/lose count 0.5/0. Average score (-1.0)
Win/lose count 1.0/3.0. Average score (-1.25)
Win/lose count 0.5/4.0. Average score (-1.7)
Win/lose count 1.0/0. Average score (-1.25)
Win/lose count 0.5/1.0. Average score (-1.1428571428571428)
Win/lose count 0/0. Average score (-1.0)
Win/lose count 1.0/3.0. Average score (-1.1111111111111112)
Win/lose count 0.5/4.0. Average score (-1.35)
Win/lose count 1.5/0. Average score (-1.0909090909090908)
Win/lose count 2.0/4.0. Average score (-1.1666666666666667)
Win/lose count 1.0/3.0. Average score (-1.2307692307692308)
Win/lose count 0.5/4.0. Average score (-1.3928571428571428)
Win/lose count 1.0/3.0. Average score (-1.4333333333333333)
Win/lose count 1.0/4.0. Average score (-1.53125)
Win/lose count 1.0/4.0. Average score (-1.6176470588235294)
Win/lose count 1.0/4.0. Average score (-1.6944444444444444)
Win/lose count 1.5/2.0. Average score (-1

In [88]:
HTML(display_videos('cnn_test0.mp4'))

In [90]:
HTML(display_videos('fc_test0.mp4'))

**Observations**:  For low values of $\epsilon$, meaning that the random exploration is rarely chosen, we observe on both videos that the rat often  gets stuck after a while and cycle within a small set of states. This usually happens when the rat has already enjoyed the rewards in his area and is surrouned by $0$ rewards values. In this area, since the rat rarely randomly explores and no moves return higher rewards, the rat is encouraged by the algorithm to stay within his region. This explains the fact that it is more pronounced at low temperatures. Setting a high $\epsilon$ value (around 0.7) is a possible solution to the problem but the agent then partially loses the ability of seeking positive rewards(cheese). Again, a good tradeoff needs to be found. To conclude, a higher $\epsilon$ value is recommanded at low temperatures since the rat has a higher risk of getting stuck and the value can be reduced for higher temperatures.

As expected, the temperature significantly affects the score. In general, the CNN performs better than the FC network. Setting a small value for the temperature (around 0.1) often leads to negative scores for both the CNN and the FC network, as both agents struggle finding the few positive rewards. Hence, the difference of performance is less obvious at low temperatures. On the other hand, we get better results with a higher temperature (around 0.7) and the CNN stands out more. The training suggests that the CNN quicker learns to avoid poisonous cells but has more difficulties finding positive rewards.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
# We take the same functions and classes as above and we modified them to tackle both issues
def train_explore(agent,env,epoch,epsilon_decay,prefix=''):
    score = 0
    loss = 0
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            
            # Get the reaction of the environment to the action
            prev_state = state
            state, reward, game_over = env.act(action, train=True)
            
            # Update gains and losses
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
        
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Decrease the exploration rate after each epoch
        agent.set_epsilon(agent.epsilon * epsilon_decay)
        
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            
        # Update stats
        score += win-lose
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.1f}/{:.1f} ({})"
            .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        # board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the rat
        self.x = 0
        self.y = 1
        
        # self time
        self.t = 0
        
        self.scale=16
        
        self.to_draw = np.zeros((max_time + 2, grid_size * self.scale, grid_size*self.scale, 3))
      
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        
        reward = 0

        # Exploration malus
        # Decrease the rewards of already visited positions
        if train:
            reward = -self.malus_position[self.x, self.y]
        

        self.malus_position[self.x, self.y] = 0.1 
        
        # Reward
        reward = reward + self.board[self.x, self.y]

        self.t = self.t + 1
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        # We added a feature containing wether the states were visited before
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

     

In [105]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.1, epsilon = 0.5, memory_size=2000, batch_size =64,n_state=3)
train_explore(agent, env, epochs_train, epsilon_decay=0.95, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore0.mp4'))

Epoch 000/020 | Loss 0.0231 | Win/lose count 7.5/24.40000000000003 (-16.90000000000003)
Epoch 001/020 | Loss 0.1411 | Win/lose count 8.4/22.900000000000052 (-14.500000000000052)
Epoch 002/020 | Loss 0.2122 | Win/lose count 5.6000000000000005/23.600000000000062 (-18.00000000000006)
Epoch 003/020 | Loss 0.0588 | Win/lose count 13.4/23.60000000000001 (-10.200000000000008)
Epoch 004/020 | Loss 0.0125 | Win/lose count 5.500000000000001/22.500000000000053 (-17.000000000000053)
Epoch 005/020 | Loss 0.0085 | Win/lose count 8.600000000000003/26.000000000000057 (-17.400000000000055)
Epoch 006/020 | Loss 0.6983 | Win/lose count 9.100000000000001/25.600000000000062 (-16.50000000000006)
Epoch 007/020 | Loss 1.5251 | Win/lose count 7.300000000000002/22.000000000000043 (-14.700000000000042)
Epoch 008/020 | Loss 0.0426 | Win/lose count 11.200000000000005/24.300000000000036 (-13.100000000000032)
Epoch 009/020 | Loss 1.3569 | Win/lose count 15.600000000000009/20.100000000000023 (-4.500000000000014)
Epoc

In [106]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 4.5/2.0. Average score (2.5)
Win/lose count 6.0/4.0. Average score (2.25)
Win/lose count 6.5/4.0. Average score (2.3333333333333335)
Win/lose count 5.5/3.0. Average score (2.375)
Win/lose count 7.0/4.0. Average score (2.5)
Win/lose count 4.0/3.0. Average score (2.25)
Win/lose count 4.5/1.0. Average score (2.4285714285714284)
Win/lose count 3.0/2.0. Average score (2.25)
Win/lose count 6.0/3.0. Average score (2.3333333333333335)
Win/lose count 4.5/1.0. Average score (2.45)
Win/lose count 8.5/4.0. Average score (2.6363636363636362)
Win/lose count 5.5/4.0. Average score (2.5416666666666665)
Win/lose count 7.5/3.0. Average score (2.6923076923076925)
Win/lose count 4.0/3.0. Average score (2.5714285714285716)
Win/lose count 10.5/4.0. Average score (2.8333333333333335)
Win/lose count 4.0/2.0. Average score (2.78125)
Win/lose count 4.5/1.0. Average score (2.823529411764706)
Win/lose count 5.5/1.0. Average score (2.9166666666666665)
Win/lose count 7.5/1.0. Average score (3.1052631

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***